In [87]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
sns.set()
sns.set_context('talk')

import warnings
warnings.filterwarnings('ignore')

import patsy
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import ttest_ind, chisquare, normaltest

In [88]:
#Read in dataset
df = pd.read_csv('homocide_database 1980-2014.csv')
df.head(5)

,Record ID,Agency Code,Agency Name,Agency Type,City,State,Year,Month,Incident,Crime Type,...,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
0,1,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,January,1,Murder or Manslaughter,...,Unknown,Male,15,Native American/Alaska Native,Unknown,Acquaintance,Blunt Object,0,0,FBI
1,2,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
2,3,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,March,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,0,FBI
3,4,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,1,Murder or Manslaughter,...,Unknown,Male,42,White,Unknown,Acquaintance,Strangulation,0,0,FBI
4,5,AK00101,Anchorage,Municipal Police,Anchorage,Alaska,1980,April,2,Murder or Manslaughter,...,Unknown,Unknown,0,Unknown,Unknown,Unknown,Unknown,0,1,FBI


In [89]:
#Sort out Texas state
df_texas = df[df['State']=="Texas"]
month_mapping = {
    'January': 1,
    'February': 2,
    'March': 3,
    'April': 4,
    'May': 5,
    'June': 6,
    'July': 7,
    'August': 8,
    'September': 9,
    'October': 10,
    'November': 11,
    'December': 12
}

df_texas['Month'] = df_texas['Month'].map(month_mapping)

In [95]:
#Specify crime type as murder
df_texas = df_texas[df_texas['Crime Type'].str.contains('Murder or Manslaughter')]
#Drop agency column
df_texas.drop(columns=['Agency Code', 'Agency Name', 'Agency Type'], inplace=True)
df_texas

,Record ID,City,State,Year,Month,Incident,Crime Type,Crime Solved,Victim Sex,Victim Age,...,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count,Record Source
19625,19626,Anderson,Texas,1980,11,1,Murder or Manslaughter,Yes,Male,19,...,Not Hispanic,Male,22,Black,Not Hispanic,Stranger,Handgun,0,0,FBI
19626,19627,Anderson,Texas,1980,11,2,Murder or Manslaughter,Yes,Female,27,...,Not Hispanic,Male,28,White,Not Hispanic,Stranger,Rifle,0,0,FBI
19627,19628,Anderson,Texas,1980,11,3,Murder or Manslaughter,Yes,Male,38,...,Not Hispanic,Male,63,White,Not Hispanic,Friend,Rifle,0,0,FBI
19629,19630,Andrews,Texas,1980,6,1,Murder or Manslaughter,Yes,Female,48,...,Unknown,Male,52,White,Unknown,Wife,Handgun,0,0,FBI
19630,19631,Andrews,Texas,1980,5,1,Murder or Manslaughter,Yes,Female,36,...,Hispanic,Male,29,White,Hispanic,Wife,Handgun,0,0,FBI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637547,637548,Bexar,Texas,2014,12,9,Murder or Manslaughter,No,Female,15,...,Hispanic,Unknown,0,Unknown,Unknown,Unknown,Strangulation,0,0,FBI
637548,637549,Bexar,Texas,2014,12,10,Murder or Manslaughter,No,Male,47,...,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0,FBI
637549,637550,Bexar,Texas,2014,12,11,Murder or Manslaughter,Yes,Female,28,...,Hispanic,Male,31,White,Not Hispanic,Girlfriend,Handgun,0,0,FBI
637550,637551,Bexar,Texas,2014,12,12,Murder or Manslaughter,Yes,Female,44,...,Hispanic,Male,43,White,Hispanic,Girlfriend,Handgun,0,0,FBI
